In [ ]:
# import libraries
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import pandas as pd
import numpy as np
import seaborn as sn

In [ ]:
# import dataset
df=pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")
df.head()

Selling_Price is the column trying to predict

In [ ]:
df.shape

In [ ]:
# getting overview of the data
df.info()

In [ ]:
# print unique value of categorical varaibles which has object data type
print(df['Fuel_Type'].unique())
print(df['Seller_Type'].unique())
print(df['Owner'].unique())
print(df['Transmission'].unique())

In [ ]:
# remove indentifiers
df_required=df.drop("Car_Name", axis=1)

In [ ]:
# Age of car
df_required["Present_Year"]=2021
df_required.head()


In [ ]:
df_required["Age_Year"]=df_required["Present_Year"]-df_required["Year"]
df_required.head()

In [ ]:
df_required.drop(["Present_Year", "Year"],axis=1, inplace = True)
df_required.head()

We need to encode data into the form of 0 and 1 to feed it into algorithm, so:

In [ ]:
df_required=pd.get_dummies(df_required, drop_first=True)
df_required.head()

In [ ]:
# extracting x variables
df_required_x=df_required.drop("Selling_Price", axis=1)

In [ ]:
# Outlier analysis:
from scipy.stats import zscore
df_required['Zscore_SP']=zscore(df_required.Selling_Price)
df_required[(df_required.Zscore_SP>3.0) | (df_required.Zscore_SP<-3.0)]


stores Zscore greater than 3.0 and less than -3.0

In [ ]:
import matplotlib.pyplot as plt
plt.boxplot(df_required["Selling_Price"])

Here, circles represents the outliers

Existence of high corelation between independent variables is called Multi Collinearity. Presence of multi collinearity can distabilize regression models. So it is better to identify multi collinearity and remove them from data appropiately. Here, I am using figure and heat map function in matplotlib library to plot correlation between independent variables in a heatmap.

In [ ]:
plt.figure(figsize=(12, 10))
sn.heatmap(df_required_x.corr(), annot=True)

If we look at this heatmap we can see one case where correlation number is high between fuel_type petrol and fuel_type diesel

In [ ]:
# train and test data
import statsmodels.api as sm

In [ ]:
x=sm.add_constant(df_required_x)
y=df_required.iloc[:, 0]


In [ ]:
import sklearn 
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y=train_test_split(x, y, train_size=0.8, random_state=100)

OLS, ordinary list square method is now used to create model.

In [ ]:
# model creation
model_1=sm.OLS(train_y, train_x).fit()
model_1.summary2()

Here, what we can see is as a speciality of multiple linear regression, R-squared value ranges from 0 to 1. It explains the variation of y i.e. Selling_Price is explained by the model. The more our R square is better is the model. We can aslo the p-value. P-value less than 0.05, means the variable is significant to the model. Another thing here is to notice is the coefficinet value and its sign which tells how the features influence the selling price. For example, we can say the kilometer driven has negative sign which means inversely proportional i.e. as the kilometer driven goes down, selling price goes up, etc.

In [ ]:
# taking only significant variable and storing to new variable
sig_var=["Present_Price", "Age_Year", "Seller_Type_Individual", "Transmission_Manual"]

In [ ]:
train_x=train_x[sig_var]

In [ ]:
Model_2=sm.OLS(train_y, train_x).fit()

In [ ]:
Model_2.summary2()

Again we can see one variable called Seller_Type_Individual is not significant so removing it.

In [ ]:
sig_var_2=["Present_Price", "Age_Year", "Transmission_Manual"]

In [ ]:
train_x=train_x[sig_var_2]

In [ ]:
Model_3=sm.OLS(train_y, train_x).fit()

In [ ]:
Model_3.summary2()

Model validation should be done now. The reason behind it is, we perform multiple linear regression under some assumptions. If this assumption is not satisfied, model cannot be reliable. 

In [ ]:
# normal distribution residuals for checking assumption 
# If the residuals are normally distributed the the blue line and red line should be close enough
probplot=sm.ProbPlot(Model_3.resid)
plt.figure(figsize=(8,6))
probplot.ppplot(line='45')
plt.show()

In [ ]:
# Predictions
pred=Model_3.predict(test_x[sig_var_2])

In [ ]:
pred

In [ ]:
# Measuring R2
from sklearn import metrics
np.round(metrics.r2_score(pred, test_y), 2)

Here we can see R2 decresed drastically. This means model need to be improved

In [ ]:
# distribution plot between predicted value and actual value
sn.distplot(test_y-pred)

In [ ]:
plt.scatter(test_y, pred)

We can see graph is linear but when the value increses accuracy goes down.